In [ ]:
%pip install stanza
%pip install farasapy
%pip install camel-tools
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 33.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
import os
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your camel_tools directory
camel_tools_dir = '/content/drive/My Drive/camel_tools/'

# Create the camel_tools directory if it doesn't exist
if not os.path.exists(camel_tools_dir):
    os.makedirs(camel_tools_dir)


os.environ['CAMELTOOLS_DATA'] = '/content/drive/My Drive/camel_tools'

!export | camel_data -i all
from camel_tools.utils.charmap import CharMapper
z
sentence = "ذهبت إلى المكتبة."
print(sentence)

ar2bw = CharMapper.builtin_mapper('ar2bw')

sent_bw = ar2bw(sentence)
print(sent_bw)


sentence = "هل ذهبت إلى المكتبة؟"
print(sentence)

# Normalize alef variants to 'ا'
sent_norm = normalize_alef_ar(sentence)
print(sent_norm)

# Normalize alef maksura 'ى' to yeh 'ي'
sent_norm = normalize_alef_maksura_ar(sent_norm)
print(sent_norm)

# Normalize teh marbuta 'ة' to heh 'ه'
sent_norm = normalize_teh_marbuta_ar(sent_norm)
print(sent_norm)
sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = sentence.split()
print(sent_split)
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer

# First, we need to load a morphological database.
# Here, we load the default database which is used for analyzing
# Modern Standard Arabic.
db = MorphologyDB.builtin_db()

analyzer = Analyzer(db)

analyses = analyzer.analyze('موظف')

for analysis in analyses:
    print(analysis, '\n')
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer

# Load the MLE disambiguator
mle = MLEDisambiguator.pretrained()

# Create a tagger using the MLE disambiguator
tagger = DefaultTagger(mle, 'pos')

# The tagger expects pre-tokenized text
sentence1 = simple_word_tokenize('نجح بايدن في الانتخابات')

# Perform POS tagging on the sentence
pos_tags = tagger.tag(sentence1)

# Print the words along with their POS tags
for word, pos_tag in zip(sentence1, pos_tags):
    print(f'{word}: {pos_tag}')


ner = NERecognizer.pretrained()

# NERecognizer expects pre-tokenized text
sentence2 = simple_word_tokenize('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع.')

labels = ner.predict_sentence(sentence2)

# Print each token paired with it's NER label
print(list(zip(sentence2, labels)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import sys
import nltk
import stanza
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from farasa.stemmer import FarasaStemmer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger
from camel_tools.ner import NERecognizer

# List of Arabic negation markers
negation_markers = ['لا', 'لم', 'ليس', 'ما', 'غير']

# Regular expression patterns to match casualties and injured
casualty_pattern = r'(?:مقتل|وفاة)\s*(\d+)|\b(استشهد|استشهدت|استشهدوا|استشهاد|قتل|قتلت|قتلوا|قتل)\s*(?:العديد\sمن\s)?(?:الجنود?\sالإسرائيليين?|المستوطنين?)'
injured_pattern = r'(?:إصابة|جرح|وإصابة|وجرح)\s*(\d+)\s*(?:مدني|عسكري|جندي|آخر)|\bإصابة(?:هم)?(?:\sالعشرات)?\b|\bأصيب(?:ت|وا|ا)\s(?:العشرات?\s)?الفلسطينيين?\b|\bجرح(?:ى|ي)\s(?:العشرات?\s)?الفلسطينيين?\b|\bمصاب(?:ون|ة|ين)?\b|\bإصابات? بالمواجهات?\b|\bأصيب(?:ت|وا|ا)\b|\bمصابين? بالرصاص\b'

# Function for Arabic stop words removal
def remove_stopwords_arabic(text):
    # Tokenization
    tokens = word_tokenize(str(text))
    # Load Arabic stopwords
    arabic_stopwords = set(stopwords.words('arabic'))
    # Remove stop words
    filtered_tokens = [word for word in tokens if word not in arabic_stopwords]
    # Join the tokens back into a sentence
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Function for applying Farasa Arabic stemmer
def apply_farasa_stemmer(text):
    # Initializing FarasaStemmer
    stemmer = FarasaStemmer()
    # Applying here Farasa Arabic stemming
    stemmed_text = stemmer.stem(text)
    return stemmed_text

# Function for handling negation/note: check if you can do something with this later on
def handle_negation(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    negated_tokens = []
    negation_scope = False

    for token in tokens:
        if token in negation_markers:
            negation_scope = True
            negated_tokens.append('NOT_' + token)
        elif negation_scope:
            negated_tokens.append('NOT_' + token)
        else:
            negated_tokens.append(token)

    negated_text = ' '.join(negated_tokens)
    return negated_text

# Function to extract casualties and injuries from NER output
def extract_casualties_injuries_ner(entities):
    casualties = []
    injured = []
    for ent in entities:
        if ent.type == 'CARDINAL' and ('مقتل' in ent.text or 'وفاة' in ent.text):
            casualties.append(ent.text)
        elif ent.type == 'CARDINAL' and ('إصابة' in ent.text or 'جرح' in ent.text):
            injured.append(ent.text)
    return casualties, injured

# Function to extract casualties and injuries using regular expressions
def extract_casualties_injuries_regex(text):
    casualties = [match[0] or match[1] for match in re.findall(casualty_pattern, text) if match[0] or match[1]]
    injured = re.findall(injured_pattern, text)
    return casualties, injured

# Function to extract casualties and injuries using dependency parsing
def extract_casualties_injuries_dep(doc):
    casualties = []
    injured = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.deprel == "nsubj" and isinstance(word.head, stanza.models.common.doc.Word) and word.head.lemma in ["مقتل", "وفاة"]:
                casualties.append(word.head.xpos)
            if word.deprel == "nsubj" and isinstance(word.head, stanza.models.common.doc.Word) and word.head.lemma in ["إصابة", "جرح"]:
                injured.append(word.head.xpos)
    return casualties, injured

# Function to perform sentence segmentation
def segment_sentences(text):
    doc = nlp(text)
    sentences = [sentence.text for sentence in doc.sentences]
    return sentences

# Ensemble function to combine NER, regex, and dependency parsing approaches
def extract_casualties_injuries_ensemble(text, entities, doc):
    ner_casualties, ner_injured = extract_casualties_injuries_ner(entities)
    regex_casualties, regex_injured = extract_casualties_injuries_regex(text)
    dep_casualties, dep_injured = extract_casualties_injuries_dep(doc)

    casualties = list(set(ner_casualties + regex_casualties + dep_casualties))
    injured = list(set(ner_injured + regex_injured + dep_injured))

    return casualties, injured

# Read Excel file
file_path = "/content/drive/My Drive/Gaza Data/cut/Original/Data5/Gaza Data Arranged5-4.xlsx"
df = pd.read_excel(file_path)

# Load the stanza pipeline with the Arabic model
nlp = stanza.Pipeline('ar', processors='tokenize')

# Preprocess Title
df['title_preprocessed'] = df['Title'].apply(remove_stopwords_arabic)
df['title_preprocessed'] = df['title_preprocessed'].apply(apply_farasa_stemmer)
df['title_preprocessed'] = df['title_preprocessed'].apply(handle_negation)
df['segmented_title'] = df['title_preprocessed'].apply(segment_sentences)

# Preprocess Text
df['text_preprocessed'] = df['Text'].apply(remove_stopwords_arabic)
df['text_preprocessed'] = df['text_preprocessed'].apply(apply_farasa_stemmer)
df['text_preprocessed'] = df['text_preprocessed'].apply(handle_negation)
df['segmented_text'] = df['text_preprocessed'].apply(segment_sentences)

# Stanza NLP pipeline
nlp = stanza.Pipeline('ar', processors='tokenize,mwt,pos,ner,lemma,depparse')

# Initialize Camel Tools tagger and NER
tagger = DefaultTagger(mle, 'pos')
ner = NERecognizer.pretrained()

# POS Tagging and NER Recognition for Title
df['title_tokens'] = df['title_preprocessed'].apply(simple_word_tokenize)
df['title_pos_tags'] = df['title_preprocessed'].apply(lambda x: [tag[1] for tag in tagger.tag(simple_word_tokenize(x))])
df['title_ner'] = df['title_preprocessed'].apply(lambda x: [tag if isinstance(tag, str) else tag[1] for tag in ner.predict_sentence(simple_word_tokenize(x))])

# Combine Title Tokens, POS Tags, NER Tags into single column 'title_preprocessed2'
df['title_preprocessed2'] = df.apply(lambda x: ' '.join(['{}-{}-{}'.format(token, pos, ner) for token, pos, ner in zip(x['title_tokens'], x['title_pos_tags'], x['title_ner'])]), axis=1)

# POS Tagging and NER Recognition for Segmented Text
df['segmented_text_tokens'] = df['segmented_text'].apply(lambda x: [simple_word_tokenize(sentence) for sentence in x])
df['segmented_text_pos_tags'] = df['segmented_text'].apply(lambda x: [[tag[1] for tag in tagger.tag(simple_word_tokenize(sentence))] for sentence in x])
df['segmented_text_ner'] = df['segmented_text'].apply(lambda x: [[tag if isinstance(tag, str) else tag[1] for tag in ner.predict_sentence(simple_word_tokenize(sentence))] for sentence in x])

# Combine Segmented Text Tokens, POS Tags, NER Tags into single column 'segmented_text_preprocessed'
df['segmented_text_preprocessed'] = df.apply(lambda x: [' '.join(['{}-{}-{}'.format(token, pos, ner) for token, pos, ner in zip(tokens, pos_tags, ner_tags)]) for tokens, pos_tags, ner_tags in zip(x['segmented_text_tokens'], x['segmented_text_pos_tags'], x['segmented_text_ner'])], axis=1)

# Extract casualties and injuries using ensemble approach
df['casualties'], df['injuries'] = zip(*df.apply(lambda row: extract_casualties_injuries_ensemble(row['text_preprocessed'], nlp(row['text_preprocessed']).ents, nlp(row['text_preprocessed'])), axis=1))

# Save to Excel with new columns
output_file_path = "/content/drive/My Drive/Gaza Data/cut/Preprocessed/Data Preprocessed5/Data Arranged5-4-Preprocessed.xlsx"
df.to_excel(output_file_path, index=False)

print("Processing completed. Data saved to:", output_file_path)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
Some weights of the model checkpoint at /content/drive/My Drive/camel_tools/data/ner/arabert were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceCla

Processing completed. Data saved to: /content/drive/My Drive/Gaza Data/cut/Preprocessed/Data Preprocessed5/Data Arranged5-4-Preprocessed.xlsx
